In [1]:
import sys
import os
import xml.etree.ElementTree as ET
import re
import STree
from tqdm import tqdm 

In [3]:
def load_all_ids():
    with open('process_results/all_ids.txt') as f:  
        lines = f.readlines()
        return lines

In [5]:
in_dir = './search_result'
dirs = os.listdir(in_dir)
out_dir = './criteria'

all_ids = load_all_ids()

for the_id in all_ids:
    xml_path = the_id.strip() + '.xml'
    print(xml_path)
    tree = ET.parse(os.path.join(in_dir, xml_path))
    
    inc_out_path = out_dir + '/' + xml_path[:-4] + '.inc'
    exc_out_path = out_dir + '/' + xml_path[:-4] + '.exc'
    
    inc_out_file = open(inc_out_path, 'w')
    exc_out_file = open(exc_out_path, 'w')
    
    for elem in tree.iter():
        if(elem.tag == "eligibility"):
            child = elem.find('.//criteria/textblock')
            if(child.text != None):
                temp_str = child.text
                temp_str = temp_str.replace("&quot;","\"")
                temp_str = temp_str.replace("&lt;","<")
                temp_str = temp_str.replace("&gt;",">")
                temp_str = temp_str.replace("&amp;","&")
                temp_str = temp_str.replace("≤", "<=")
                temp_str = temp_str.replace("≥",">=")
                temp_str = temp_str.replace("°","*")
                temp_str = temp_str.replace("℃","*C")
                temp_str = temp_str.replace("μ","micro-")
                st_include = re.search("Inclusion Criteria", temp_str,re.IGNORECASE)
                print("=======Inclusion Criteria=======")
                if(st_include is not None):
                    st_include = st_include.start()
                    st_exclude = re.search("Exclusion Criteria", temp_str,re.IGNORECASE)
                    if(st_exclude is not None):
                        st_exclude = st_exclude.start()
                        in_str = temp_str[st_include+19:st_exclude].encode('ascii','ignore')
                        ex_str = temp_str[st_exclude+19:].encode('ascii','ignore')
                        # criteria_split(in_str, "in", xml_path, csv_writer)
                        inc_out_file.write(''.join(chr(x) for x in in_str))
                        print("=======Exclusion Criteria=======")
                        # criteria_split(ex_str, "ex", xml_path, csv_writer)
                        exc_out_file.write(''.join(chr(x) for x in ex_str))
                    else:
                        in_str = temp_str[st_include+19:].encode('ascii','ignore')
                        # criteria_split(in_str, "in", xml_path, csv_writer)
                        inc_out_file.write(''.join(chr(x) for x in in_str))
                else:
                    # criteria_split(temp_str, "in", xml_path, csv_writer) 
                    # dirs = os.listdir(args.in_dir)
                    inc_out_file.write(''.join(chr(x) for x in temp_str.encode('ascii','ignore')))
    inc_out_file.close()
    exc_out_file.close()

NCT02735707.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT03331445.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT03680274.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT03808922.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04244591.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04251871.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04252274.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04252664.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04254874.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04255017.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04257656.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04260594.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04261270.xml


=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04327401.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04327505.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328012.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328272.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328285.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328441.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328467.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328480.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04328493.xml
=======Inclusion Criteria=======
NCT04328961.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04329195.xml
=======Inclusion Criteria=======
=======Exclusion Criteria=======
NCT04329572.xml
=======Inclusion Criteria=======
=======Exclusion

In [6]:
def cleanup(s1):
    return ' '.join([ts for ts in [t.strip('.,:-()\n\t') for t in s1.split(' ')] if ts != '' and len(ts) > 1])

def jaccard_score_levmatch(s1, s2, theta = 0.9, steps = 1):

    # clean up strings
    ts1 = [ts for ts in [t.strip('.,:-') for t in s1.split(' ')] if ts != '' and len(ts) > 1]
    ts2 = [ts for ts in [t.strip('.,:-') for t in s2.split(' ')] if ts != '' and len(ts) > 1]

    i = 0
    #u = len(ts1) + len(ts2)
    u = sum([len(t) for t in ts1]) + sum([len(t) for t in ts2])
    # print(u)

    for t1 in ts1:
        for t2 in ts2:
            if levmatch(t1, t2, theta = theta, steps = steps):
                # print(t1)
                avglen = int((sum([len(t) for t in t1]) + sum([len(t) for t in t2])) / 2) 
                i += avglen
                u -= avglen
                ts2.remove(t2)
                # print(ts2)
                break
    return i/u
  
def levmatch(s, t, theta = 0.67, steps = 6):
    ratio = levenshtein_ratio(s, t)
    edits = levenshtein_distance(s, t)
    return ratio > theta or (min(len(s), len(t)) > steps and edits < steps) # match!

def levenshtein_ratio(s, t):
    return levenshtein_ratio_and_distance(s, t, True)

def levenshtein_distance(s, t):
    return levenshtein_ratio_and_distance(s, t, False)

def levenshtein_ratio_and_distance(s, t, ratio_calc = True):
    rows = len(s)+1
    cols = len(t)+1
    # distance = np.zeros((rows,cols),dtype = int)
    distance = [[0 for r in range(cols+1)] for c in range(rows+1)]

    # Populate matrix of zeros with the indeces of each character of both strings
    for i in range(1, rows):
        for k in range(1,cols):
            distance[i][0] = i
            distance[0][k] = k

    # Iterate over the matrix to compute the cost of deletions,insertions and/or substitutions    
    for col in range(1, cols):
        for row in range(1, rows):
            if s[row-1] == t[col-1]:
                cost = 0 # If the characters are the same in the two strings in a given position [i,j] then the cost is 0
            else:
                # In order to align the results with those of the Python Levenshtein package, if we choose to calculate the ratio
                # the cost of a substitution is 2. If we calculate just distance, then the cost of a substitution is 1.
                if ratio_calc == True:
                    cost = 2
                else:
                    cost = 1
            distance[row][col] = min(distance[row-1][col] + 1,      # Cost of deletions
                                 distance[row][col-1] + 1,          # Cost of insertions
                                 distance[row-1][col-1] + cost)     # Cost of substitutions
    if ratio_calc == True:
        # Computation of the Levenshtein Distance Ratio
        Ratio = ((len(s)+len(t)) - distance[row][col]) / (len(s)+len(t))
        return Ratio
    else:
        # print(distance) # Uncomment if you want to see the matrix showing how the algorithm computes the cost of deletions,
        # insertions and/or substitutions
        # This is the minimum number of edits needed to convert string a to string b
        # return "The strings are {} edits away".format(distance[row][col])
        return distance[row][col]



### refactor function

In [7]:
def load_inc_pattern_label():
    pattern_label_list = []
    with open('./COVID-19-ProcV0/incPatterns-labeled.txt', 'r') as patfile:
        pat_label = patfile.readline().strip('\n').split('\t') # header
        lines = 1
        while lines < 407: 
            pat_label = patfile.readline().strip('\n').split('\t')
            pattern_label_list.append(pat_label)
            lines += 1
        return pattern_label_list

def load_exc_pattern_label():
    pattern_label_list = []
    with open('./COVID-19-ProcV0/excPatterns-labeled.txt', 'r') as patfile:
        pat_label = patfile.readline().strip('\n').split('\t') # header
        lines = 1
        while lines < 557: 
            pat_label = patfile.readline().strip('\n').split('\t')
            pattern_label_list.append(pat_label)
            lines += 1
        return pattern_label_list

In [8]:
def get_labels_with_criteria(c,pattern_labels):
    label_seq = []
    
    if len(c) == 0:
        return label_seq
    
    
    st = STree.STree(c)
    
    for pat_label in pattern_labels:
        matches = st.find_all(pat_label[0])
        if matches != []:
            label_seq = label_seq + [[start, pat_label[0], pat_label[1]] for start in matches]
    
    label_seq.sort()
    
    # remove duplicates
    previous = -1
    for i in range(len(label_seq)):
        if i>0 and label_seq[i][0] == label_seq[i-1][0]:
            if len(label_seq[i][1]) > previous:
                label_seq[i-1][0] = -1
                previous = len(label_seq[i][1])
            else:
                label_seq[i][0] = -1
        else:
            previous = len(label_seq[i][1])

    label_seq = [labsq for labsq in label_seq if labsq[0] > 0]

    return label_seq

In [9]:
def get_label_from_file(file,pattern_label):
    if os.path.isfile(file):
        with open(file,'r') as f:
            crit_text = f.read()
            a = cleanup(crit_text).lower()
            label_list = get_labels_with_criteria(a,pattern_label)
            return label_list
    else:
        print(file + ' does not exist.')

In [10]:
def get_all_criteria():
    all_ids = load_all_ids()
    all_contents = []
    
    dataset_path = './criteria'
    for the_id in all_ids:
        the_id = the_id.strip()
        print(the_id)
        file1 = the_id + '.inc'
        file2 = the_id + '.exc'
        file_path_1 = os.path.join(dataset_path,file1)
        file_path_2 = os.path.join(dataset_path,file2)
        
        inc_lab = get_label_from_file(file_path_1,inc_pattern_label)
        exc_lab = get_label_from_file(file_path_2,exc_pattern_label)
        
        content = (the_id,inc_lab,exc_lab)
        all_contents.append(content)

    return all_contents

In [11]:
exc_pattern_label = load_exc_pattern_label()
inc_pattern_label = load_inc_pattern_label()

content_list = get_all_criteria()

NCT02735707
NCT03331445
NCT03680274
NCT03808922
NCT04244591
NCT04251871
NCT04252274
NCT04252664
NCT04254874
NCT04255017
NCT04257656
NCT04260594
NCT04261270
NCT04261426
NCT04261517
NCT04261907
NCT04263402
NCT04264533
NCT04264858
NCT04268537
NCT04273321
NCT04273529
NCT04273581
NCT04273646
NCT04273763
NCT04275245
NCT04275388
NCT04275414
NCT04276688
NCT04278963
NCT04279197
NCT04280588
NCT04280705
NCT04285190
NCT04286503
NCT04287686
NCT04288713
NCT04290858
NCT04290871
NCT04291053
NCT04291729
NCT04292730
NCT04292899
NCT04293887
NCT04295551
NCT04298060
NCT04302766
NCT04303299
NCT04303507
NCT04304053
NCT04304313
NCT04305106
NCT04305457
NCT04306393
NCT04306497
NCT04306705
NCT04307693
NCT04308317
NCT04308668
NCT04310228
NCT04310865
NCT04311177
NCT04311697
NCT04312009
NCT04312243
NCT04312997
NCT04313023
NCT04315298
NCT04315480
NCT04315896
NCT04315948
NCT04316377
NCT04317040
NCT04317092
NCT04318015
NCT04318444
NCT04320238
NCT04320277
NCT04320615
NCT04321096
NCT04321174
NCT04321278
NCT04321616
NCT0

In [12]:
for tp in content_list:
    nct_id = tp[0]
    file_path = './labels/' + tp[0] + '.txt'
    
    with open(file_path,'w') as f:
        for i in tp[1]:
            f.write("%s\n" % i)
        
        f.write("\n")
        
        for i in tp[2]:
            f.write("%s\n" % i)
        
        f.write("\n")